In [2]:
import pandas as pd
from tqdm import tqdm
import math

In [15]:
#!pip list
#!python -m spacy download es_core_news_sm

In [2]:
data = pd.read_csv('./training_data/cel_ner_100.csv')
data.head() 

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600,Nokia,9.0,14.0
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6,Motorola,8.0,16.0
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb,iPhone,0.0,6.0
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray,iPhone,1.0,7.0
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite,Huawei,0.0,6.0


## Format Data

In [3]:
TRAINING_DATA = []

for index, row in data.iterrows(): 
    text = row['Title']
    start = row['start']
    end = row['end']
    entities = []
    #print(start)
    if math.isnan(start):
        print('no ent')
    else:        
        entities.append((int(start), int(end), "MARCA"))            
        
    
    #print(dup_start)
    training_example = (text, {"entities": entities})
    TRAINING_DATA.append(training_example)

#print(TRAINING_DATA)

no ent
no ent
no ent
no ent
no ent


In [4]:
len(TRAINING_DATA)

100

In [5]:
import spacy
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
from tqdm import tqdm
#spacy.prefer_gpu()
#TRAIN_DATA = [
#    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
#    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
#]

#nlp = spacy.blank("en")
nlp = spacy.load('es_core_news_sm', disable = ['tagger', 'ner'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1500000
docs = []
for text, annot in tqdm(TRAINING_DATA):
    if len(text) < 1000000:
        doc = nlp(text)
        tags = biluo_tags_from_offsets(doc, annot['entities'])
        entities = spans_from_biluo_tags(doc, tags)
        doc.ents = entities
        docs.append(doc)

srsly.write_json("./training_data/cel_ner_100.json", [docs_to_json(docs)]) 

100%|██████████| 100/100 [00:00<00:00, 775.07it/s]


# Debug

In [6]:
!python3 -m spacy debug-data es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_100.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_100.json' --verbose


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'es'
100 training docs
100 evaluation docs
⚠ 99 training examples also in evaluation data
⚠ Low number of examples to train from a blank model (100)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 698 total words in the data (233 unique)
10 most common words: 'Gb' (37), 'iPhone' (30), 'Celular' (28), 'Samsung' (28),
'gb' (28), 'Galaxy' (17), '128' (16), 'Motorola' (15), 'Ram' (15), 'Plus' (14)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 1 new label, 0 existing labels
0 missing values (tokens with '-' label)
New: 'MARCA' (95)
✔ Good amount of examples for all labels
✔ Examples 

# Train

In [7]:
!python -m spacy train es NUBI-CEL-NER-100 '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_100.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_100.json' --base-model 'es_core_news_sm' -p ner -R
#SAco el modelo de base
#!python -m spacy train en NER-SECOND-EMPTY-VEC-15000 '/home/fede/kaggle_competition_2/spacy_2.3.5/data_for_training/train_sent_12500_format.json' '/home/fede/kaggle_competition_2/spacy_2.3.5/data_for_training/dev_sent_3100_format.json' --vectors '/home/fede/kaggle_competition_2/spacy_2.3.5/Diego/w2v-w11-f7-50-spacy'  -p ner -R

✔ Created output directory: NUBI-CEL-NER-100
Training pipeline: ['ner']
Starting with base model 'es_core_news_sm'
Replacing component from base model 'ner'
Counting training words (limit=0)
/home/fede/anaconda3/envs/spacy_env/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(

Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  1    492.271   0.000   0.000   0.000  100.000    33629                        
  2    189.516   0.000   0.000   0.000  100.000    33429                        
  3     68.739  97.778  92.632  95.135  100.000    31409                        
  4     13.250  97.938  100.000  98.958  100.000    31267                       
  

## Celular Inference 

In [8]:
import spacy
from spacy import displacy

nlp = spacy.load('./models/NUBI-CEL-NER-100/model-best')


In [9]:
evaluate = pd.read_csv('../../Celulares.csv') #/home/fede/kaggle/desafo_ia_nlp_nubimetrics/
evaluate.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite


In [10]:
marca = []
start = []
end = []

for doc in nlp.pipe(evaluate['Title'][:1000]):    
    #displacy.render(doc, style="ent")
    ent_count = 0
    for ent in doc.ents:
        #print(doc.text, ent.text, ent.start_char, ent.end_char)
        if ent_count == 1:
            pass
        else:        
            marca.append(ent.text)
            start.append(ent.start_char)
            end.append(ent.end_char)
            ent_count = 1
    if ent_count == 0:    
        marca.append('##REV')
        start.append('##REV')
        end.append('##REV')


In [11]:
len(marca)

1000

In [12]:
df_1000 = evaluate.loc[:999, :].copy()
df_1000['marca'] = marca
df_1000['start'] = start
df_1000['end'] = end

In [13]:
#df_1000.to_csv("./training_data/cel_ner_textcat_1000_corregir.csv", index=False)

## Celular data 1000 examples

In [3]:
data = pd.read_csv('./training_data/cel_ner_textcat_1000.csv')
data.head() 

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600,Nokia,9.0,14.0
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6,Motorola,8.0,16.0
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb,iPhone,0.0,6.0
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray,iPhone,1.0,7.0
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite,Huawei,0.0,6.0


In [5]:
TRAINING_DATA = []

for index, row in data.iterrows(): 
    text = row['Title']
    start = row['start']
    end = row['end']
    entities = []
    #print(start)
    if math.isnan(start):
        pass
        #print('no ent')
    else:        
        entities.append((int(start), int(end), "MARCA"))            
        
    
    #print(dup_start)
    training_example = (text, {"entities": entities})
    TRAINING_DATA.append(training_example)

print(len(TRAINING_DATA))

1000


In [6]:
import random
#TRAINING_DATA
random.seed(0)
l = list(TRAINING_DATA)
random.shuffle(l)
#print(l)
length = len(TRAINING_DATA)
index_80 = length*0.80
train = l[:int(index_80)]
dev = l[int(index_80):]
print(len(train))
print(len(dev))

800
200


In [7]:
import spacy
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
from tqdm import tqdm


nlp = spacy.load('es_core_news_sm', disable = ['tagger', 'ner'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1500000

def spacy_format(data, test_train):    
    docs = []
    for text, annot in tqdm(data):
        if len(text) < 1000000:
            doc = nlp(text)
            tags = biluo_tags_from_offsets(doc, annot['entities'])
            entities = spans_from_biluo_tags(doc, tags)
            doc.ents = entities
            docs.append(doc)

    srsly.write_json(f'./training_data/cel_ner_1000_{test_train}.json', [docs_to_json(docs)]) 

In [8]:
spacy_format(train, 'train')
spacy_format(dev, 'dev')

 47%|████▋     | 377/800 [00:00<00:00, 768.77it/s]/tmp/ipykernel_11533/1580151283.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Vendo Cel.samsung S21 Ultra 256 G Super Impecable" with entities "[(10, 17, 'MARCA')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tags = biluo_tags_from_offsets(doc, annot['entities'])
 66%|██████▋   | 532/800 [00:00<00:00, 756.55it/s]/tmp/ipykernel_11533/1580151283.py:16: UserWarning: [W030] Some entities could not be aligned in the text "H6iPhone XS Max 256gb. Año 2020. En Caja. Impecabl..." with entities "[(2, 8, 'MARCA')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tags = biluo_tags_from_offsets(doc, annot['entities'])
 86%|████████▌ | 688/800 [00:00<00:00, 765.28it/s]/tmp/i

# Debug

In [9]:
!python3 -m spacy debug-data es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1000_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1000_dev.json' --verbose


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'es'
800 training docs
200 evaluation docs
⚠ 10 training examples also in evaluation data
⚠ Low number of examples to train from a blank model (800)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 5767 total words in the data (880 unique)
10 most common words: 'Gb' (326), 'iPhone' (266), 'Celular' (212), 'gb' (210),
'Samsung' (204), '64' (127), 'Ram' (124), '128' (119), 'Plus' (102), 'Galaxy'
(101)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 1 new label, 0 existing labels
0 missing values (tokens with '-' label)
New: 'MARCA' (727)
✔ Good amount of examples for all labels
✔ Exa

# Train 1000 data

In [10]:
!python -m spacy train es NUBI-CEL-NER-1000 '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1000_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1000_dev.json' --base-model 'es_core_news_sm' -p ner -R

✔ Created output directory: NUBI-CEL-NER-1000
Training pipeline: ['ner']
Starting with base model 'es_core_news_sm'
Replacing component from base model 'ner'
Counting training words (limit=0)
/home/fede/anaconda3/envs/spacy_env/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(

Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  1    903.482  98.889  97.268  98.072  100.000    31849                        
  2     52.391  98.361  98.361  98.361  100.000    32107                        
  3     18.722  98.361  98.361  98.361  100.000    32274                        
  4     12.614  98.361  98.361  98.361  100.000    31801                        
 

# Inference 1000

In [11]:
import spacy
from spacy import displacy

nlp = spacy.load('./models/NUBI-CEL-NER-1000/model-best')

In [12]:
evaluate = pd.read_csv('../../Celulares.csv')
evaluate.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite


In [13]:
marca = []
start = []
end = []

for doc in nlp.pipe(evaluate['Title']):    
    #displacy.render(doc, style="ent")
    ent_count = 0
    for ent in doc.ents:
        #print(doc.text, ent.text, ent.start_char, ent.end_char)
        if ent_count == 1:
            pass
        else:        
            marca.append(ent.text)
            start.append(ent.start_char)
            end.append(ent.end_char)
            ent_count = 1
    if ent_count == 0:    
        marca.append('##REV')
        start.append('##REV')
        end.append('##REV')

In [14]:
df_all = evaluate.loc[:, :].copy()
df_all['marca'] = marca
df_all['start'] = start
df_all['end'] = end

In [15]:
df_all.to_csv("./bd_cel_ner_all.csv", index=False)

## Celular data 1000 more 187 examples (Corregidos luego de la inferencia)

In [16]:
data = pd.read_csv('./training_data/cel_ner_textcat_1000_more_187.csv')
data.head() 

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title,marca,start,end
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600,Nokia,9.0,14.0
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6,Motorola,8.0,16.0
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb,iPhone,0.0,6.0
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray,iPhone,1.0,7.0
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite,Huawei,0.0,6.0


In [17]:
TRAINING_DATA = []

for index, row in data.iterrows(): 
    text = row['Title']
    start = row['start']
    end = row['end']
    entities = []
    #print(start)
    if math.isnan(start):
        pass
        #print('no ent')
    else:        
        entities.append((int(start), int(end), "MARCA"))            
        
    
    #print(dup_start)
    training_example = (text, {"entities": entities})
    TRAINING_DATA.append(training_example)

print(len(TRAINING_DATA))

1187


In [18]:
import random
#TRAINING_DATA
random.seed(0)
l = list(TRAINING_DATA)
random.shuffle(l)
#print(l)
length = len(TRAINING_DATA)
index_80 = length*0.80
train = l[:int(index_80)]
dev = l[int(index_80):]
print(len(train))
print(len(dev))

949
238


In [20]:
import spacy
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
from tqdm import tqdm


nlp = spacy.load('es_core_news_sm', disable = ['tagger', 'ner'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 1500000

def spacy_format(data, test_train):    
    docs = []
    for text, annot in tqdm(data):
        if len(text) < 1000000:
            doc = nlp(text)
            tags = biluo_tags_from_offsets(doc, annot['entities'])
            entities = spans_from_biluo_tags(doc, tags)
            doc.ents = entities
            docs.append(doc)

    srsly.write_json(f'./training_data/cel_ner_1187_{test_train}.json', [docs_to_json(docs)])

In [21]:
spacy_format(train, 'train')
spacy_format(dev, 'dev')

  8%|▊         | 73/949 [00:00<00:01, 726.66it/s]/tmp/ipykernel_11533/798063398.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Celular Hello Kitty Kity Libre Llamada & Sms Radio..." with entities "[(8, 20, 'MARCA')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tags = biluo_tags_from_offsets(doc, annot['entities'])
 16%|█▌        | 150/949 [00:00<00:01, 748.72it/s]/tmp/ipykernel_11533/798063398.py:16: UserWarning: [W030] Some entities could not be aligned in the text "Vendo Lgk8 Liberado Sin Detalles" with entities "[(6, 8, 'MARCA')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  tags = biluo_tags_from_offsets(doc, annot['entities'])
 66%|██████▋   | 629/949 [00:00<00:00, 799.58it/s]/tmp/ipykernel_11533/798063

In [22]:
# Debug
!python3 -m spacy debug-data es '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1187_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1187_dev.json' --verbose


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'es'
949 training docs
238 evaluation docs
⚠ 16 training examples also in evaluation data
⚠ Low number of examples to train from a blank model (949)
It's recommended to use at least 2000 examples (minimum 100)

============================== Vocab & Vectors ==============================
ℹ 7062 total words in the data (1077 unique)
10 most common words: 'Gb' (344), 'Celular' (327), 'iPhone' (278), 'gb' (256),
'Samsung' (210), '64' (135), 'Ram' (132), '128' (123), 'Galaxy' (109), '32'
(106)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 1 new label, 0 existing labels
0 missing values (tokens with '-' label)
New: 'MARCA' (841)
⚠ 1 entity span(s) with punctuation
✔ Good amou

In [23]:
# Train 1000 data
!python -m spacy train es NUBI-CEL-NER-1187 '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1187_train.json' '/home/fede/kaggle/desafio_ia_nlp_nubimetrics/spacy_ner/cel/training_data/cel_ner_1187_dev.json' --base-model 'es_core_news_sm' -p ner -R

✔ Created output directory: NUBI-CEL-NER-1187
Training pipeline: ['ner']
Starting with base model 'es_core_news_sm'
Replacing component from base model 'ner'
Counting training words (limit=0)
/home/fede/anaconda3/envs/spacy_env/lib/python3.8/site-packages/spacy/language.py:635: UserWarning: [W033] Training a new parser or NER using a model with an empty lexeme normalization table. This may degrade the performance to some degree. If this is intentional or this language doesn't have a normalization table, please ignore this warning.
  proc.begin_training(

Itn  NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  ------  ------  ------  -------  -------
  1    970.669  90.909  92.233  91.566  100.000    33468                        
  2    102.070  95.098  94.175  94.634  100.000    33393                        
  3     56.529  97.537  96.117  96.822  100.000    33301                        
  4     29.847  95.238  97.087  96.154  100.000    34092                        
 

In [28]:
# Inference 1000
import spacy
from spacy import displacy

nlp = spacy.load('./models/NUBI-CEL-NER-1187/model-best')
evaluate = pd.read_csv('../../Celulares.csv')
evaluate.head()

,Unnamed: 0,Encrypted_Seller,Encrypted_Id,Title
0,0,7a30f2b511ff323b31dc01de07736128b49d5ec4c50b01...,054dd4f1afba59722078d8a4af2f1b3742d767e836ace2...,Teléfono Nokia 1600
1,1,f6fed6d5cf9a02286d81f9c842a3874511ff1d3d411a94...,6c6a089073a7ba0ba6897eb51b0a0b5dc50d8071edafef...,Celular Motorola G6
2,2,eef17e4b9fa3ccc0232a39061e31e721f5ca6a7a44b786...,78364fc1d8574957fe3508ed687d862a0bbcf5e5136558...,iPhone 6 Plus 16gb
3,3,2ab9d5c1ec1dc9cf166fd20e51a3e734ab5076a4e5c73d...,896a81cc2da0ecb382e3b519783a8ef2d9ee0ce3872ea3...,iPhone 8 64 Gb Space Gray
4,4,0f9b41b18f3c3776dc9515e1eb2a8686d4b271341444bb...,e6733e9d8b3bd38642d841c4509b0a5d6b5545d9a75fab...,Huawei Mate 10 Lite


In [25]:
marca = []
start = []
end = []

for doc in nlp.pipe(evaluate['Title']):    
    #displacy.render(doc, style="ent")
    ent_count = 0
    for ent in doc.ents:
        #print(doc.text, ent.text, ent.start_char, ent.end_char)
        if ent_count == 1:
            pass
        else:        
            marca.append(ent.text)
            start.append(ent.start_char)
            end.append(ent.end_char)
            ent_count = 1
    if ent_count == 0:    
        marca.append('##REV')
        start.append('##REV')
        end.append('##REV')

In [26]:
df_all = evaluate.loc[:, :].copy()
df_all['marca'] = marca
df_all['start'] = start
df_all['end'] = end

In [27]:
df_all.to_csv("./final_bd_cel_ner_all.csv", index=False)